删掉pdf中参考文献部分，pdf太长会导致kimi反应慢，想本地读pdf可以参考这边，结合问ds完善代码

In [53]:
from openai import OpenAI
import pandas as pd
import openpyxl
import json
import os
import time
import shutil
from pathlib import Path
import fitz  # PyMuPDF
import re

In [54]:

def load_info(file_name):
    try:
        with open(file_name + ".txt", "r", encoding='utf-8') as file:
            info = file.read().strip()
        return info
    except FileNotFoundError:
        raise Exception("{}.txt not found. Please create it.".format(file_name))
    except Exception as e:
        raise Exception(f"Error reading info: {str(e)}")

request = load_info("MXenes/request")

        
def trim_pdf_after_references(pdf_path, clean_filename):
    doc = fitz.open(pdf_path)      
    output_path =  os.path.join("trimm", clean_filename[:20] + "_MXenes.pdf")
    
    total_pages = len(doc)
    reference_page  = -1
    

    # 优先查找中文的“参考文献”
    for page_num in range(total_pages):
        page = doc[page_num]
        text = page.get_text("text")
        if "参考文献" in text:
            reference_page = page_num
            found_keyword = "参考文献"
            break

    # 如果中文关键词未找到，则尝试查找英文的“References”
    if reference_page == -1:
        for page_num in range(total_pages):
            page = doc[page_num]
            text = page.get_text("text")
            if "References" in text or "REFERENCES" in text:
                reference_page = page_num
                found_keyword = "References"
                break

    # 定义输出路径，并确保目录存在
    output_path = os.path.join("trimm", clean_filename[:20] + "_MXenes.pdf")
    os.makedirs(os.path.dirname(output_path), exist_ok=True) # 确保输出目录存在

    if reference_page == -1:
        print(f"未找到 '参考文献' 或 'References' 部分在 '{pdf_path}'。保存完整 PDF 到: '{output_path}'")
    else:
        print(f"在 '{pdf_path}' 中找到 '{found_keyword}' 部分，从第 {reference_page + 1} 页开始删除。")
        # 删除从参考文献页到文档末尾的所有页面
        doc.delete_pages(from_page=reference_page, to_page=total_pages-1)
        print(f"修剪后的 PDF 从 {total_pages} 页到 {len(doc)} 页。")
    
    # 统一保存文档
    doc.save(output_path)
    doc.close()
    print(f"修剪后的文件保存到: '{output_path}'")



In [55]:

pdf_folder = "MXenes/9篇文献"
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith('.pdf'):
        print(f"正在处理文件: {pdf_file}") # 以便跟踪进度
        pdf_path = os.path.join(pdf_folder, pdf_file)
        output_sheet = os.path.splitext(pdf_file)[0][:15]
        num = pdf_file[:2].split('.')[0]
        print(num)

        clean_filename = re.sub(r'\(.*?\)', '', pdf_file)

        try:
            trim_pdf_after_references(pdf_path, clean_filename)
        except Exception as e:
            print(f"处理文件 '{pdf_file}' 时出错: {e}")
            print(f"请检查文件 '{pdf_file}' 是否损坏或格式异常。")
            continue # 继续处理下一个文件

正在处理文件: 1永定河门头沟段抗生素污染特征及风险评估.pdf
1永
在 'MXenes/9篇文献\1永定河门头沟段抗生素污染特征及风险评估.pdf' 中找到 '参考文献' 部分，从第 6 页开始删除。
修剪后的 PDF 从 7 页到 5 页。
修剪后的文件保存到: 'trimm\1永定河门头沟段抗生素污染特征及风险评估_MXenes.pdf'
正在处理文件: 2抗生素在城市河流中的污染特征及生态毒性研究_卫毅梅.pdf
2抗
在 'MXenes/9篇文献\2抗生素在城市河流中的污染特征及生态毒性研究_卫毅梅.pdf' 中找到 '参考文献' 部分，从第 10 页开始删除。
修剪后的 PDF 从 78 页到 9 页。
修剪后的文件保存到: 'trimm\2抗生素在城市河流中的污染特征及生态毒性_MXenes.pdf'
正在处理文件: 3高效液相色谱-电喷雾串联质谱法检测环境水样中22种抗生素类药物.pdf
3高
在 'MXenes/9篇文献\3高效液相色谱-电喷雾串联质谱法检测环境水样中22种抗生素类药物.pdf' 中找到 '参考文献' 部分，从第 7 页开始删除。
修剪后的 PDF 从 7 页到 6 页。
修剪后的文件保存到: 'trimm\3高效液相色谱-电喷雾串联质谱法检测环境_MXenes.pdf'
正在处理文件: 4高效液相色谱-串联质谱法同时测定地表水中33种药物.pdf
4高
在 'MXenes/9篇文献\4高效液相色谱-串联质谱法同时测定地表水中33种药物.pdf' 中找到 '参考文献' 部分，从第 9 页开始删除。
修剪后的 PDF 从 12 页到 8 页。
处理文件 '4高效液相色谱-串联质谱法同时测定地表水中33种药物.pdf' 时出错: code=7: object out of range (0 0 R); xref size 75
请检查文件 '4高效液相色谱-串联质谱法同时测定地表水中33种药物.pdf' 是否损坏或格式异常。
正在处理文件: 5Pharmaceuticals and personal care products.pdf
5P
在 'MXenes/9篇文献\5Pharmaceuticals and personal care products.pdf' 中找到 'References' 部分，从第 6 页开始删除。
修剪